# Final code

In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import time 
import torch
import pickle
from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.nn import GATv2Conv,GATConv
import torch.nn.functional as F
from sklearn.metrics import precision_score, accuracy_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
# for GAT run this else run next block
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads = 3):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels,heads = heads)
        self.conv2 = GATConv(hidden_channels*heads, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=100
):
    best_model = model
    best_val_auc = 0
    val_auc_list = []
    train_loss_list = []
    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)
        val_auc_list.append(val_auc)
        if (val_auc > best_val_auc):
            
            torch.save(model,"best_link_prediction.pt")
            best_model = model
            best_val_auc = val_auc
            line = "Epoch: "+str(epoch) + "\tTrain Loss: "+str(loss) + "\tVal AUC: "+str(best_val_auc)
            
        if epoch%10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")
        #train_loss_list.append({loss:.3f})
        train_loss_list.append(loss)
    print("\n\n for best model :\n",line)
    
    return best_model,val_auc_list,train_loss_list #,train_auc_list


@torch.no_grad()
def eval_link_predictor(model, data):
    with torch.no_grad():
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [3]:
#user set params
dir_path = "WSJ split data files"
dbname = "WSJ"
node_feature_dir_path = "WSJ node features"
epochs = 100

In [4]:
for split_no in range(1,2,1):
    test_file_no = split_no
    print("Test no ", test_file_no, " going on")
    print("\n\n\n Test file no : ",test_file_no)
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
    
    # collecting all positive edges
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
    final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  
    
    # downloading  node_embeddings 
    node_embeddings = pickle.load(open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","rb"))
    
    #making training and val graphs
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)

    split = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0)
    train_data, val_data, test_data = split(data)
    
    del train_df, node_embeddings, final_edge_list, data
    
    print(train_data)
    print(val_data)
    print(test_data)
    break
    model = Net(8, 3, 2) #.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    curr = time.time()
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=epochs)
    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    torch.save(best_model.state_dict(), "my_"+str(dbname)+"_GAT_model_state_dict_"+str(test_file_no))
    torch.save(best_model, "my_"+str(dbname)+"_GAT_whole_model_"+str(test_file_no))

Test no  1  going on



 Test file no :  1


FileNotFoundError: [Errno 2] No such file or directory: 'WSJ split data files/WSJ_positive_edges_split_2.csv'

# GATv2

In [3]:
# for GAT run this else run next block
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads = 3):
        super().__init__()
        self.conv1 = GATv2Conv(in_channels, hidden_channels,heads = heads)
        self.conv2 = GATv2Conv(hidden_channels*heads, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=100
):
    best_model = model
    best_val_auc = 0
    val_auc_list = []
    train_loss_list = []
    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)
        val_auc_list.append(val_auc)
        if (val_auc > best_val_auc):
            
            torch.save(model,"best_link_prediction.pt")
            best_model = model
            best_val_auc = val_auc
            line = "Epoch: "+str(epoch) + "\tTrain Loss: "+str(loss) + "\tVal AUC: "+str(best_val_auc)
            
        if epoch%10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")
        #train_loss_list.append({loss:.3f})
        train_loss_list.append(loss)
    print("\n\n for best model :\n",line)
    
    return best_model,val_auc_list,train_loss_list #,train_auc_list


@torch.no_grad()
def eval_link_predictor(model, data):
    with torch.no_grad():
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [4]:
for split_no in range(7,11,1):
    test_file_no = split_no
    print("Test no ", test_file_no, " going on")
    print("\n\n\n Test file no : ",test_file_no)
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
    
    # collecting all positive edges
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
    final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  
    
    # downloading  node_embeddings 
    node_embeddings = pickle.load(open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","rb"))
    
    #making training and val graphs
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)

    split = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0)
    train_data, val_data, test_data = split(data)
    
    del train_df, node_embeddings, final_edge_list, data
    
    model = Net(8, 3, 2) #.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    curr = time.time()
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=epochs)
    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    torch.save(best_model.state_dict(), "my_"+str(dbname)+"_WSJ_GATv2_model_state_dict_"+str(test_file_no))
    torch.save(best_model, "my_"+str(dbname)+"_WSJ_GATv2_whole_model_"+str(test_file_no))
    
    del model, train_data, val_data

Test no  1  going on



 Test file no :  1
Epoch: 010, Train Loss: 0.672, Val AUC: 0.845
Epoch: 020, Train Loss: 0.645, Val AUC: 0.817
Epoch: 030, Train Loss: 0.620, Val AUC: 0.837
Epoch: 040, Train Loss: 0.597, Val AUC: 0.836
Epoch: 050, Train Loss: 0.582, Val AUC: 0.833
Epoch: 060, Train Loss: 0.572, Val AUC: 0.830
Epoch: 070, Train Loss: 0.565, Val AUC: 0.833
Epoch: 080, Train Loss: 0.560, Val AUC: 0.835
Epoch: 090, Train Loss: 0.556, Val AUC: 0.836
Epoch: 100, Train Loss: 0.554, Val AUC: 0.836


 for best model :
 Epoch: 11	Train Loss: tensor(0.6695, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)	Val AUC: 0.8463994362303524
time taken for training :  191.62732458114624
time taken for per epoch :  1.9162732458114624
Test no  2  going on



 Test file no :  2
Epoch: 010, Train Loss: 0.981, Val AUC: 0.803
Epoch: 020, Train Loss: 1.035, Val AUC: 0.617
Epoch: 030, Train Loss: 0.835, Val AUC: 0.714
Epoch: 040, Train Loss: 0.734, Val AUC: 0.810
Epoch: 050, Train Loss: 0.711, Val AUC: 0.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/shellygupta/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_780058/3482021667.py", line 47, in <module>
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=epochs)
  File "/tmp/ipykernel_780058/892779624.py", line 40, in train_link_predictor
    neg_edge_index = negative_sampling(
  File "/home/shellygupta/.local/lib/python3.8/site-packages/torch_geometric/utils/negative_sampling.py", line 107, in negative_sampling
    return vector_to_edge_index(neg_idx, size, bipartite, force_undirected)
  File "/home/shellygupta/.local/lib/python3.8/site-packages/torch_geometric/utils/negative_sampling.py", line 371, in vector_to_edge_index
    col[row <= col] += 1
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last

TypeError: object of type 'NoneType' has no len()

In [5]:
for split_no in range(7,11,1):
    test_file_no = split_no
    print("Test no ", test_file_no, " going on")
    print("\n\n\n Test file no : ",test_file_no)
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
    
    # collecting all positive edges
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
    final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  
    
    # downloading  node_embeddings 
    node_embeddings = pickle.load(open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","rb"))
    
    #making training and val graphs
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)

    split = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0)
    train_data, val_data, test_data = split(data)
    
    del train_df, node_embeddings, final_edge_list, data
    
    model = Net(8, 3, 2) #.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    curr = time.time()
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=epochs)
    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    torch.save(best_model.state_dict(), "my_"+str(dbname)+"_WSJ_GATv2_model_state_dict_"+str(test_file_no))
    torch.save(best_model, "my_"+str(dbname)+"_WSJ_GATv2_whole_model_"+str(test_file_no))
    
    del model, train_data, val_data

Test no  7  going on



 Test file no :  7
Epoch: 010, Train Loss: 190.579, Val AUC: 0.504
Epoch: 020, Train Loss: 140.922, Val AUC: 0.507
Epoch: 030, Train Loss: 85.733, Val AUC: 0.513
Epoch: 040, Train Loss: 53.159, Val AUC: 0.557
Epoch: 050, Train Loss: 22.183, Val AUC: 0.613
Epoch: 060, Train Loss: 14.089, Val AUC: 0.631
Epoch: 070, Train Loss: 9.815, Val AUC: 0.632
Epoch: 080, Train Loss: 7.869, Val AUC: 0.640
Epoch: 090, Train Loss: 6.624, Val AUC: 0.651
Epoch: 100, Train Loss: 5.647, Val AUC: 0.661


 for best model :
 Epoch: 100	Train Loss: tensor(5.6474, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)	Val AUC: 0.6608748369205648
time taken for training :  183.69712805747986
time taken for per epoch :  1.8369712805747986
Test no  8  going on



 Test file no :  8
Epoch: 010, Train Loss: 0.855, Val AUC: 0.559
Epoch: 020, Train Loss: 0.721, Val AUC: 0.723
Epoch: 030, Train Loss: 0.667, Val AUC: 0.789
Epoch: 040, Train Loss: 0.651, Val AUC: 0.772
Epoch: 050, Train Loss: 0.638, Va

In [6]:
for split_no in [7,9,10]:
    test_file_no = split_no
    print("Test no ", test_file_no, " going on")
    print("\n\n\n Test file no : ",test_file_no)
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
    
    # collecting all positive edges
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
    final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  
    
    # downloading  node_embeddings 
    node_embeddings = pickle.load(open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","rb"))
    
    #making training and val graphs
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)

    split = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0)
    train_data, val_data, test_data = split(data)
    
    del train_df, node_embeddings, final_edge_list, data
    
    model = Net(8, 3, 2) #.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    curr = time.time()
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=epochs)
    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    torch.save(best_model.state_dict(), "my_"+str(dbname)+"_last_WSJ_GATv2_model_state_dict_"+str(test_file_no))
    torch.save(best_model, "my_"+str(dbname)+"_last_WSJ_GATv2_whole_model_"+str(test_file_no))
    
    del model, train_data, val_data

Test no  7  going on



 Test file no :  7
Epoch: 010, Train Loss: 0.720, Val AUC: 0.623
Epoch: 020, Train Loss: 0.687, Val AUC: 0.666
Epoch: 030, Train Loss: 0.677, Val AUC: 0.739
Epoch: 040, Train Loss: 0.672, Val AUC: 0.820
Epoch: 050, Train Loss: 0.668, Val AUC: 0.844
Epoch: 060, Train Loss: 0.662, Val AUC: 0.850
Epoch: 070, Train Loss: 0.654, Val AUC: 0.853
Epoch: 080, Train Loss: 0.632, Val AUC: 0.862
Epoch: 090, Train Loss: 0.607, Val AUC: 0.866
Epoch: 100, Train Loss: 0.591, Val AUC: 0.867


 for best model :
 Epoch: 86	Train Loss: tensor(0.6139, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)	Val AUC: 0.8678133643194659
time taken for training :  184.82191967964172
time taken for per epoch :  1.8482191967964172
Test no  9  going on



 Test file no :  9
Epoch: 010, Train Loss: 12.625, Val AUC: 0.491
Epoch: 020, Train Loss: 9.137, Val AUC: 0.469
Epoch: 030, Train Loss: 6.394, Val AUC: 0.520
Epoch: 040, Train Loss: 4.275, Val AUC: 0.535
Epoch: 050, Train Loss: 2.816, Val AUC: 0

KeyboardInterrupt: 

In [5]:
for split_no in [9,10]:
    test_file_no = split_no
    print("Test no ", test_file_no, " going on")
    print("\n\n\n Test file no : ",test_file_no)
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
    
    # collecting all positive edges
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
    final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  
    
    # downloading  node_embeddings 
    node_embeddings = pickle.load(open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","rb"))
    
    #making training and val graphs
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)

    split = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0)
    train_data, val_data, test_data = split(data)
    
    del train_df, node_embeddings, final_edge_list, data
    
    model = Net(8, 3, 2) #.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    curr = time.time()
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=epochs)
    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    torch.save(best_model.state_dict(), "my_"+str(dbname)+"_last_WSJ_GATv2_model_state_dict_"+str(test_file_no))
    torch.save(best_model, "my_"+str(dbname)+"_last_WSJ_GATv2_whole_model_"+str(test_file_no))
    
    del model, train_data, val_data

Test no  9  going on



 Test file no :  9
Epoch: 010, Train Loss: 0.681, Val AUC: 0.745
Epoch: 020, Train Loss: 0.607, Val AUC: 0.870
Epoch: 030, Train Loss: 0.576, Val AUC: 0.866
Epoch: 040, Train Loss: 0.559, Val AUC: 0.861
Epoch: 050, Train Loss: 0.548, Val AUC: 0.861
Epoch: 060, Train Loss: 0.541, Val AUC: 0.863
Epoch: 070, Train Loss: 0.537, Val AUC: 0.866
Epoch: 080, Train Loss: 0.536, Val AUC: 0.867
Epoch: 090, Train Loss: 0.534, Val AUC: 0.869
Epoch: 100, Train Loss: 0.533, Val AUC: 0.870


 for best model :
 Epoch: 21	Train Loss: tensor(0.6026, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)	Val AUC: 0.8698653630849256
time taken for training :  188.95362830162048
time taken for per epoch :  1.8895362830162048
Test no  10  going on



 Test file no :  10
Epoch: 010, Train Loss: 663.670, Val AUC: 0.500
Epoch: 020, Train Loss: 537.002, Val AUC: 0.500
Epoch: 030, Train Loss: 431.600, Val AUC: 0.500
Epoch: 040, Train Loss: 345.106, Val AUC: 0.500
Epoch: 050, Train Loss: 274.962,

In [5]:
for split_no in [10]:
    test_file_no = split_no
    print("Test no ", test_file_no, " going on")
    print("\n\n\n Test file no : ",test_file_no)
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        
            
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    
    #making node_id variables continuous
    temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
    
    # collecting all positive edges
    train_df['src'] = [temp_node_no[i] for i in train_df['src']]
    train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
    final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
    final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  
    
    # downloading  node_embeddings 
    node_embeddings = pickle.load(open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","rb"))
    
    #making training and val graphs
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)

    split = T.RandomLinkSplit(
        num_val=0.05,
        num_test=0.0,
        is_undirected=True,
        add_negative_train_samples=False,
        neg_sampling_ratio=1.0)
    train_data, val_data, test_data = split(data)
    
    del train_df, node_embeddings, final_edge_list, data
    
    model = Net(8, 3, 2) #.to(device)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    criterion = torch.nn.BCEWithLogitsLoss()
    
    curr = time.time()
    best_model,_,_ = train_link_predictor(model, train_data, val_data, optimizer, criterion,n_epochs=150)
    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/150)
    
    torch.save(best_model.state_dict(), "my_"+str(dbname)+"_last_WSJ_GATv2_model_state_dict_"+str(test_file_no))
    torch.save(best_model, "my_"+str(dbname)+"_last_WSJ_GATv2_whole_model_"+str(test_file_no))
    
    del model, train_data, val_data

Test no  10  going on



 Test file no :  10
Epoch: 010, Train Loss: 0.623, Val AUC: 0.859
Epoch: 020, Train Loss: 0.600, Val AUC: 0.864
Epoch: 030, Train Loss: 0.575, Val AUC: 0.866
Epoch: 040, Train Loss: 0.555, Val AUC: 0.867
Epoch: 050, Train Loss: 0.541, Val AUC: 0.869
Epoch: 060, Train Loss: 0.534, Val AUC: 0.869
Epoch: 070, Train Loss: 0.532, Val AUC: 0.869
Epoch: 080, Train Loss: 0.531, Val AUC: 0.870
Epoch: 090, Train Loss: 0.532, Val AUC: 0.870
Epoch: 100, Train Loss: 0.530, Val AUC: 0.871
Epoch: 110, Train Loss: 0.530, Val AUC: 0.871
Epoch: 120, Train Loss: 0.530, Val AUC: 0.872
Epoch: 130, Train Loss: 0.530, Val AUC: 0.872
Epoch: 140, Train Loss: 0.530, Val AUC: 0.872
Epoch: 150, Train Loss: 0.530, Val AUC: 0.872


 for best model :
 Epoch: 150	Train Loss: tensor(0.5299, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)	Val AUC: 0.8722163302107182
time taken for training :  293.67836475372314
time taken for per epoch :  1.957855765024821


# create node features

In [4]:
temp_node_no = pickle.load(open("node_id_into_continuous_node_ids.p","rb"))
for split_no in range(1,11,1):
    test_file_no = split_no
    node_embeddings = pd.read_csv(node_feature_dir_path + "/node_features_excluding_split_"+str(test_file_no)+".csv")
    node_embeddings["node_id"] = [temp_node_no[i] for i in node_embeddings["node_id"]]
    node_embeddings = node_embeddings.sort_values(by = 'node_id')
    column = "1"
    node_embeddings[column] = (node_embeddings[column] - node_embeddings[column].mean()) / node_embeddings[column].std() 
    column = "2"
    node_embeddings[column] = (node_embeddings[column] - node_embeddings[column].mean()) / node_embeddings[column].std() 
    column = "3"
    node_embeddings[column] = (node_embeddings[column] - node_embeddings[column].mean()) / node_embeddings[column].std() 
    column = "12"
    node_embeddings[column] = (node_embeddings[column] - node_embeddings[column].mean()) / node_embeddings[column].std() 
    column = "32"
    node_embeddings[column] = (node_embeddings[column] - node_embeddings[column].mean()) / node_embeddings[column].std()
    
    node_embeddings = node_embeddings.set_index('node_id')
    df = node_embeddings.copy()
    node_features = []
    for index, row in df.iterrows():
        node_features.append(torch.Tensor([list(row)]))

    node_features = torch.cat(tuple(node_features),dim=0)  
    
    pickle.dump(node_features,open(node_feature_dir_path + "/node_embeddings_for_pytorch_models_excluding_split_"+str(test_file_no)+".p","wb"))
    del node_embeddings,node_features,df
    
    #making graph
    data = Data()
    data.x = node_embeddings
    data.edge_index = final_edge_list.type(torch.int64)


# rough work

In [ ]:
import pandas as pd
import torch
import torch_geometric
import numpy as np
import random#user set params
dir_path = "WSJ split data files"
dbname = "WSJ"
epochs = 100

#variables required
flatten_file_name = 'WSJ_comm_flatten.csv'
dbname = "WSJ"

#read flatten file
df = pd.read_csv(flatten_file_name) 
df = df.drop(['time', 'article_id',"comment_id"], axis=1)

# converting nodes to continuous variable
temp_node_no = {}
id_to_node = {}
id = 0
for i in np.unique(df[['src', 'dest']].values):
    temp_node_no[i] = id
    id_to_node[id] = i
    id += 1
print("total unique nodes found : ",id)

# collecting all positive edges
df['new_src'] = [temp_node_no[i] for i in df['src']]
df['new_dest'] = [temp_node_no[i] for i in df['dest']]
positive_edges = [torch.Tensor([[df['new_src'][i],df['new_dest'][i]]]) for i in df.index]
positive_edges = torch.cat(tuple(positive_edges),dim=0)
print("len of df : ",len(df))
print("len of edges before removing duplicates : ",len(positive_edges))

#checking for and removing duplicates
positive_edges = torch.unique(positive_edges,dim=0)
print("Total unique edges : ",len(positive_edges))

# diving positive into 10 split
df = df.drop(['new_src', 'new_dest'], axis=1)                   
df["label"] = [1 for i in range(len(df))]
df = df.sample(frac=1).reset_index(drop=True)
positive_split = np.array_split(df, 10)
for i in range(len(positive_split)):
    file_name = str(dbname)+"_positive_edges_split_"+str(i+1)+".csv"
    positive_split[i].to_csv(file_name, encoding='utf-8', index=False)                   
print("Positive edges successfully divided into 10 partitions")
del df, positive_split

#finally generating negative edges
negative_edges = torch_geometric.utils.negative_sampling(edge_index = positive_edges.T)
negative_edges = negative_edges.tolist()
neg_df = pd.DataFrame(negative_edges, columns = ['src', 'dest'])
del negative_edges, positive_edges
neg_df["label"] = [-1 for i in range(len(neg_df))]
print("No of negative edges generated : ",len(neg_df))
                   
# converting neg_df into predecided node ids
new_src= [id_to_node[i] for i in neg_df['src']]
new_dest = [id_to_node[i] for i in neg_df['dest']]
neg_df['src'] = new_src
neg_df['dest'] = new_dest 
del new_src, new_dest                   
neg_df["weight"] = [random.randint(1,301) for counter in range(len(neg_df))]
neg_df["sign"] = [random.randint(-2,2) for counter in range(len(neg_df))]
neg_df = neg_df.loc[:,['src','dest','weight','sign','label']]  
print("Negative edges df successfully created")
                   
# diving negative edges into 10 splits
neg_df = neg_df.sample(frac=1).reset_index(drop=True)
negative_split = np.array_split(neg_df, 10)
for i in range(len(negative_split)):
    file_name = str(dbname)+"_negative_edges_split_"+str(i+1)+".csv"
    negative_split[i].to_csv(file_name, encoding='utf-8', index=False)
print("Negative edges successfully divided into 10 partitions")
del neg_df, negative_split

In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
import time 
import torch
import pickle
from torch_geometric.data import Data
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from sklearn.metrics import precision_score, accuracy_score,f1_score,roc_auc_score

In [2]:
# for GAT run this else run next block
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import negative_sampling


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads = 3):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels,heads = heads)
        self.conv2 = GATConv(hidden_channels*heads, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(
            dim=-1
        )  # product of a pair of nodes on each edge

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()
    

def train_link_predictor(
    model, train_data, val_data, optimizer, criterion, n_epochs=100
):
    best_model = model
    best_val_auc = 0
    val_auc_list = []
    train_loss_list = []
    for epoch in range(1, n_epochs + 1):

        model.train()
        optimizer.zero_grad()
        z = model.encode(train_data.x, train_data.edge_index)

        # sampling training negatives for every training epoch
        neg_edge_index = negative_sampling(
            edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
            num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

        edge_label_index = torch.cat(
            [train_data.edge_label_index, neg_edge_index],
            dim=-1,
        )
        edge_label = torch.cat([
            train_data.edge_label,
            train_data.edge_label.new_zeros(neg_edge_index.size(1))
        ], dim=0)

        out = model.decode(z, edge_label_index).view(-1)
        loss = criterion(out, edge_label)
        loss.backward()
        optimizer.step()

        val_auc = eval_link_predictor(model, val_data)
        val_auc_list.append(val_auc)
        if (val_auc > best_val_auc):
            print("best_model at epoch : ",epoch)
            torch.save(model,"best_link_prediction.pt")
            best_model = model
            best_val_auc = val_auc
            line = "Epoch: "+str(epoch) + "\tTrain Loss: "+str(loss) + "\tVal AUC: "+str(best_val_auc)
            
        if epoch%10 == 0:
            print(f"Epoch: {epoch:03d}, Train Loss: {loss:.3f}, Val AUC: {val_auc:.3f}")
        #train_loss_list.append({loss:.3f})
        train_loss_list.append(loss)
    print("\n\n for best model :\n",line)
    
    return best_model,val_auc_list,train_loss_list #,train_auc_list


@torch.no_grad()
def eval_link_predictor(model, data):
    with torch.no_grad():
        model.eval()
        z = model.encode(data.x, data.edge_index)
        out = model.decode(z, data.edge_label_index).view(-1).sigmoid()

    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())

In [3]:
dir_path = "WSJ split data files"
test_file_no = 1

In [4]:
train_df = pd.DataFrame()
test_df = pd.DataFrame()
for i in range(1,11,1):
    if i != test_file_no:
        df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
        train_df = train_df.append(df, ignore_index=True)
        del df
    else:
        test_df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv")
        neg_df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
        test_df = test_df.append(neg_df, ignore_index=True)
        del neg_df
train_df

<ipython-input-4-ae5d777dec2f>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_df = test_df.append(neg_df, ignore_index=True)
<ipython-input-4-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

,src,dest,layer,weight,sign,label
0,5046,8395,12,2,-2.0,1
1,100036,103501,3,27,-2.0,1
2,100670,102593,3,2,-2.0,1
3,870,5764,2,33,-1.0,1
4,5036,6048,12,7,-2.0,1
...,...,...,...,...,...,...
569695,100442,100451,3,4,-2.0,1
569696,5338,8134,12,3,-2.0,1
569697,100075,100086,3,5,-2.0,1
569698,5082,5400,12,2,-2.0,1


<ipython-input-4-ae5d777dec2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)


,src,dest,layer,weight,sign,label
0,5046,8395,12,2,-2.0,1
1,100036,103501,3,27,-2.0,1
2,100670,102593,3,2,-2.0,1
3,870,5764,2,33,-1.0,1
4,5036,6048,12,7,-2.0,1
...,...,...,...,...,...,...
569695,100442,100451,3,4,-2.0,1
569696,5338,8134,12,3,-2.0,1
569697,100075,100086,3,5,-2.0,1
569698,5082,5400,12,2,-2.0,1


In [5]:
train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
test_df = test_df.drop(['layer', 'weight',"sign"], axis=1)

In [6]:
# converting nodes to continuous variable
temp_node_no = {}
id = 0
node_embeddings = []
for i in np.unique(train_df[['src', 'dest']].values):
    temp_node_no[i] = id
    if temp_node_no[i]<5000:
        node_embeddings.append(torch.Tensor([[1,0,0]]))
    elif temp_node_no[i]<100000:
        node_embeddings.append(torch.Tensor([[0,1,0]]))
    else:
        node_embeddings.append(torch.Tensor([[0,0,1]]))
    id += 1
    
node_embeddings = torch.cat(tuple(node_embeddings),dim=0)  

print("total unique nodes found till train_df: ",id)

total unique nodes found till train_df:  8536
total unique nodes found till train_df:  8536


In [7]:
train_df['src'] = [temp_node_no[i] for i in train_df['src']]
train_df['dest'] = [temp_node_no[i] for i in train_df['dest']]
final_edge_list = [torch.Tensor([train_df['src']]),torch.Tensor([train_df['dest']])]
final_edge_list = torch.cat(tuple(final_edge_list),dim=0)  

In [8]:
final_edge_list

tensor([[  74., 4921., 5555.,  ..., 4960.,  110., 6270.],
        [3423., 8386., 7478.,  ..., 4971.,  428., 8120.]])

tensor([[  74., 4921., 5555.,  ..., 4960.,  110., 6270.],
        [3423., 8386., 7478.,  ..., 4971.,  428., 8120.]])

In [9]:
data = Data()
data.x = node_embeddings
data.edge_index = final_edge_list.type(torch.int64)

In [10]:
node_embeddings

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])

tensor([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        ...,
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])

In [11]:
data

Data(x=[8536, 3], edge_index=[2, 569700])

Data(x=[8536, 3], edge_index=[2, 569700])

In [12]:
split = T.RandomLinkSplit(
    num_val=0.05,
    num_test=0.0,
    is_undirected=True,
    add_negative_train_samples=False,
    neg_sampling_ratio=1.0,
)
train_data, val_data, test_data = split(data)

In [13]:
train_data

Data(x=[8536, 3], edge_index=[2, 1082430], edge_label=[541215], edge_label_index=[2, 541215])

Data(x=[8536, 3], edge_index=[2, 1082430], edge_label=[541215], edge_label_index=[2, 541215])

In [14]:
del train_df,node_embeddings,final_edge_list

In [16]:
del node_embeddings, train_df

NameError: name 'node_embeddings' is not defined

NameError: name 'node_embeddings' is not defined

In [17]:

model = Net(3, 3, 2) #.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
import time
curr = time.time()
best_model,val_auc_list,train_loss_list = train_link_predictor(model, train_data, 
                                                                              val_data, optimizer, criterion,n_epochs=250)
timetaken = time.time()-curr
print("time taken for training : ",timetaken)
print("time taken for per epoch : ",timetaken/epochs)
#test_auc = eval_link_predictor(best_model, test_data)
#print(f"Test: {test_auc:.3f}")

best_model at epoch :  1
best_model at epoch :  1
best_model at epoch :  2
best_model at epoch :  2
best_model at epoch :  3
best_model at epoch :  3
best_model at epoch :  4
best_model at epoch :  4
best_model at epoch :  5
best_model at epoch :  5
best_model at epoch :  6
best_model at epoch :  6
best_model at epoch :  7
best_model at epoch :  7
best_model at epoch :  8
best_model at epoch :  8
best_model at epoch :  9
best_model at epoch :  9
best_model at epoch :  10
Epoch: 010, Train Loss: 0.702, Val AUC: 0.289
best_model at epoch :  10
Epoch: 010, Train Loss: 0.702, Val AUC: 0.289
best_model at epoch :  11
best_model at epoch :  11
best_model at epoch :  12
best_model at epoch :  12
best_model at epoch :  13
best_model at epoch :  13
best_model at epoch :  14
best_model at epoch :  14
best_model at epoch :  15
best_model at epoch :  15
best_model at epoch :  16
best_model at epoch :  16
best_model at epoch :  17
best_model at epoch :  17
best_model at epoch :  18
best_model at ep

NameError: name 'epochs' is not defined

NameError: name 'epochs' is not defined

In [18]:
torch.save(model.state_dict(), "gat_1_params")
torch.save(model, "gat_1_for_whole_dataset")